In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install transformers datasets accelerate peft trl bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-6hgcp1m_/unsloth_4fad726c6239400f90f22082e18c65af
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-6hgcp1m_/unsloth_4fad726c6239400f90f22082e18c65af
  Resolved https://github.com/unslothai/unsloth.git to commit b1c782ec411d3c06576a8b240a07304b6266fcb7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 84.8 MB/s eta 0:00:00
 

In [ ]:
from unsloth import FastLanguageModel
import torch

MAX_SEQ_LEN = 2048
LOAD_IN_4BIT = True
# MODEL_NAME = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit"
MODEL_NAME = "unsloth/gemma-7b-it-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LEN,
    dtype=None,
    load_in_4bit=LOAD_IN_4BIT,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.1: Fast Gemma patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.57G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

# load dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

dataset = pd.read_excel('/content/drive/MyDrive/datasets/News.xlsx')
dataset.head()


,Unnamed: 0,content,title
0,0,شرح ادعا: یک کاربر شبکه اجتماعی ایکس به نقل از...,طبق قانون جدید، افرادی که تا چهل سالگی ازدواج ...
1,1,شرح ادعا: خبرگزاری تابناک در گزارشی مدعی شده ا...,کیم جونگ اون با پاسپورت جعلی به ژاپن رفته است
2,2,شرح ادعا اکانت «ویکی تجربه» در شبکه اجتماعی ای...,اعمال محدودیت سنی در آگهی‌های استخدامی در اروپ...
3,3,شرح ادعا: یک کاربر شبکه اجتماعی اینستاگرام در ...,سوخت‌رسان انگلیسی در قطر، جنگنده‌های اسرائیلی ...
4,4,شرح ادعا: یک کاربر شبکه اجتماعی ایکس در پستی م...,ترور نافرجام خالد مشعل در 1997 منجر به تضعیف د...


In [ ]:
from sklearn.model_selection import train_test_split

dataset = dataset.drop(columns=['Unnamed: 0'])
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

# Zero shot

In [ ]:
from transformers import pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto"
)

Device set to use cuda:0


In [ ]:
max_model_tokens = 1998
max_new_tokens = 50

def truncate_prompt(tokenizer, text, max_model_tokens=max_model_tokens, max_new_tokens=max_new_tokens):
    # تبدیل متن به توکن
    inputs = tokenizer(text, return_tensors="pt")
    input_ids = inputs["input_ids"]

    # اگر طول ورودی زیاد است، فقط آخر متن را نگه دار
    if input_ids.shape[-1] + max_new_tokens > max_model_tokens:
        truncated_ids = input_ids[:, -(max_model_tokens - max_new_tokens):]
        text = tokenizer.decode(truncated_ids[0], skip_special_tokens=True)

    return text


In [ ]:
import re

def clean_output_title(text):
    match = re.split(r"### title:", text, flags=re.IGNORECASE)
    if len(match) > 1:
        return match[-1].strip()
    return text.strip()


In [ ]:
# zero shot fo i-th of test dataset
def zero_shot(i):
  print("real title: " + test_data['title'].iloc[i])
  content = test_data['content'].iloc[i]

  prompt1 = f"""The text below is a news content. Your task is to write a short, clear, and relevant Persian title that summarizes the main idea of the article.
  Generate only one Persion title and do not include any translations or extra text.  News content is:\n\n"
              "{content}\n\ntitle:"""

  result = pipe(prompt1, max_new_tokens=50, temperature=0.6, do_sample=True)
  print("Result for prompt 1: " + clean_output_title(result[0]["generated_text"]))

  prompt2 = f"""Write a title in Persian for the news article below.
  Do not include any English words, translations, or extra explanations. News content is:\n\n"
              "{content}\n\ntitle:"""

  result = pipe(prompt2, max_new_tokens=50, temperature=0.6, do_sample=True)
  print("Result for prompt 2: " + clean_output_title(result[0]["generated_text"]))

  prompt3 = f"""You are a journalist. Your task is to read the news content below and write a single, clear, and concise title **in Persian** that best summarizes the main idea.
  Do not include any English words, translations, or extra text. News content is:\n\n"
              "{content}\n\ntitle:"""

  result = pipe(prompt3, max_new_tokens=50, temperature=0.6, do_sample=True)
  print("Result for prompt 3: " + clean_output_title(result[0]["generated_text"]))

zero_shot(0)
zero_shot(1)


# Few shot

In [ ]:

example_1 = f"News content:\n{train_data['content'].iloc[0]}\nTitle:\n{train_data['title'].iloc[0]}"
example_2 = f"News content:\n{train_data['content'].iloc[1]}\nTitle:\n{train_data['title'].iloc[1]}"

examples_text = f"{example_1}\n\n{example_2}\n\nNow write the title for the next news:\n\n"


def few_shot(i):
    content = test_data['content'].iloc[i]
    real_title = test_data['title'].iloc[i]

    prompt1 = f"""The text below is a news content. Your task is to write a short, clear, and relevant Persian title that summarizes the main idea of the article.
Generate only one Persian title and do not include any translations or extra text. Two prepared examples are provided below. Study them carefully and write your output in the same style.
Examples:\n
{examples_text}\n\n
News content:\n\n{content}\n\ntitle:"""
    prompt1 = truncate_prompt(tokenizer, prompt1)
    result = pipe(prompt1, max_new_tokens=50, temperature=0.6, do_sample=True)
    print("Result for prompt 1: " + clean_output_title(result[0]["generated_text"]))

    prompt2 = f"""Write a title in Persian for the news article below.
Do not include any English words, translations, or extra explanations.Two prepared examples are provided below. Study them carefully and write your output in the same style.
Examples:\n
{examples_text}\n\n
News content:\n\n{content}\n\ntitle:"""

    prompt2 = truncate_prompt(tokenizer, prompt2)
    result = pipe(prompt2, max_new_tokens=50, temperature=0.6, do_sample=True)
    print("Result for prompt 2: " + clean_output_title(result[0]["generated_text"]))

    prompt3 = f"""You are a journalist. Your task is to read the news content below and write a single, clear, and concise title **in Persian** that best summarizes the main idea.
Do not include any English words, translations, or extra text.Two prepared examples are provided below. Study them carefully and write your output in the same style.
Examples:\n
{examples_text}\n\n
News content:\n\n{content}\n\ntitle:"""

    prompt3 = truncate_prompt(tokenizer, prompt3)
    result = pipe(prompt3, max_new_tokens=50, temperature=0.6, do_sample=True)
    print("Result for prompt 3: " + clean_output_title(result[0]["generated_text"]))



few_shot(0)
few_shot(1)



# Fine Tune

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_alpha = 16,
    lora_dropout = 0.0,
    bias = "none",
    random_state=3407,
)

Unsloth 2025.11.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
from datasets import Dataset

def prepare_data(df):
    df["prompt"] = df["content"].apply(
        lambda x: f"""### Instruction:\nGenerate a short and suitable Persian title for the following news content.
         Examples for train include input and
        what you should output are here: \n\n### input:\n{x}\n\n### output:"""
    )
    df["response"] = df["title"].apply(lambda x: f" {x.strip()}")
    return df[["prompt", "response"]]


train_df = prepare_data(train_data)
test_df = prepare_data(test_data)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

from trl import SFTTrainer
from transformers import TrainingArguments
from torch.cuda import is_bf16_supported

MAX_SEQ_LEN = 2048

EOS_TOKEN = tokenizer.eos_token

def formatting_func(examples):
    texts = []
    for prompt, response in zip(examples["prompt"], examples["response"]):
        texts.append(f"{prompt.strip()} {response.strip()}{EOS_TOKEN}")
    return texts



training_args = TrainingArguments(
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 8,
    warmup_steps = 5,
    max_steps = 113,
    learning_rate = 1e-5,
    fp16 = not is_bf16_supported(),
    bf16 = is_bf16_supported(),
    logging_steps = 1,
    output_dir = "outputs",
    optim = "paged_adamw_8bit",
    save_total_limit = 2,
)


trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    max_seq_length = MAX_SEQ_LEN,
    args = training_args,
    formatting_func = formatting_func,
)

trainer.train()


In [ ]:

def generate_title_finetuned(model, tokenizer, news_text, max_len=50, temperature=0.7):
    inputs = tokenizer(news_text, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_len,
        do_sample=True,
        temperature=temperature,
        top_p=0.9,
        top_k=50,
        eos_token_id=tokenizer.eos_token_id
    )

    # title = tokenizer.decode(outputs[0], skip_special_tokens=True)
    title = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return title



In [ ]:
import re

def clean_output(text):
    match = re.split(r"### output:", text, flags=re.IGNORECASE)
    if len(match) > 1:
        return match[-1].strip()
    return text.strip()


In [ ]:


from tqdm import tqdm

generated_titles = []

for prompt in tqdm(test_df["prompt"], desc="Generating titles"):
    prompt = truncate_prompt(tokenizer, prompt)
    title = generate_title_finetuned(model, tokenizer, prompt)

    if isinstance(title, list):
        text = title[0]
    else:
        text = title

    clean_title = clean_output(text)
    generated_titles.append(clean_title)

test_df["generated_title"] = generated_titles

print(test_df[["prompt", "response", "generated_title"]].head())


In [ ]:

!pip install evaluate
!pip install rouge_score


In [ ]:
import evaluate

bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")

references = test_df["response"].tolist()
predictions = test_df["generated_title"].tolist()

bleu_result = bleu.compute(predictions=predictions, references=references)

rouge_result = rouge.compute(predictions=predictions, references=references)

meteor_result = meteor.compute(predictions=predictions, references=references)

print("BLEU:", bleu_result["bleu"])
print("ROUGE-L:", rouge_result["rougeL"])
print("METEOR:", meteor_result["meteor"])


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


BLEU: 0.10978596700493812
ROUGE-L: 0.01206896551724138
METEOR: 0.244404032466368


In [ ]:
test_df.to_csv("/content/gemma_results.csv", index=False, encoding="utf-8-sig")